In [ ]:
import pathlib
import os
import sys

PROJECT_DIRECTORY = pathlib.PurePath(os.getcwd()).parent

project_directory = str(PROJECT_DIRECTORY)
if project_directory not in sys.path:
    sys.path.insert(0, project_directory)
print(project_directory)

In [ ]:
from src.fusion_detector import perturbation, dataset

batch = dataset.NormalizedCifarBatchDataset(8).batch
x, labels = batch

image = x[7].unsqueeze(0)
print(image.requires_grad)

In [ ]:
import matplotlib.pyplot as plt
import torchvision.transforms as T

original = image.squeeze().clone().detach()

mean = (0.4914, 0.4822, 0.4465)
std = (0.2471, 0.2435, 0.2616)
for channel in range(3):
    original[channel].mul_(std[channel]).add_(mean[channel])

plt.axis("off")
plt.imshow(T.ToPILImage()(original))
plt.show()

In [ ]:
from src.fusion_detector.thirdparty.pytorch_cifar10 import module as M

vgg = M.vgg19_bn(pretrained=True)

In [ ]:
attack = perturbation.fgsm(vgg, epsilon=0.1)

adversarial = attack(image)
adversarial_show = adversarial.squeeze().clone().detach()

for channel in range(3):
    adversarial_show[channel].mul_(std[channel]).add_(mean[channel])

plt.axis("off")
plt.imshow(T.ToPILImage()(adversarial_show))
plt.show()

In [ ]:
after = image.squeeze().clone().detach()

mean = (0.4914, 0.4822, 0.4465)
std = (0.2471, 0.2435, 0.2616)
# for channel in range(3):
#     after[channel].mul_(std[channel]).add_(mean[channel])

plt.axis("off")
plt.imshow(T.ToPILImage()(after))
plt.show()

In [ ]:
import torch

assert torch.equal(original, after)

In [ ]:
difference = adversarial_show - original
plt.axis("off")
plt.imshow(T.ToPILImage()(difference))
plt.show()